# T5 TPU Tensorflow 

In [1]:
%tensorflow_version 2.x
!pip install transformers=4.3.3
!pip install sentencepiece

     |████████████████████████████████| 1.9MB 4.9MB/s 
     |████████████████████████████████| 3.2MB 22.6MB/s 
     |████████████████████████████████| 890kB 39.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=4c0430e4ed63878137560e2b64173e1b652a22f9cc653579d257d1272948670d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.2MB 5.3MB/s 


### Imports

We'll only be importing the components that we'll use during this tutorial: the TensorFlow model alongside the model specific tokenizer. The last two imports will manage the pre-processing of our data.

In [2]:
import tensorflow as tf
print(tf.__version__)

import os
from transformers import ( 
    T5Tokenizer, 
    TFT5Model, 
    TFT5ForConditionalGeneration
)

2.4.1


In [3]:
learning_rate = 3e-5

model_size = "t5-base"

BATCH_SIZE_PER_REPLICA = 6

EPOCHS = 8

MAX_ARTICLE_LEN = 512

MAX_HIGHLIGHT_LEN = 150

# Building the training system

## Strategy

We make use of TensorFlow's strategies, which handle the data distribution as well as the distributed training that happens on the devices available. In this example we'll be using a `MirroredStrategy` which can be used to train on a multiple GPUs in a distributed manner. 

In [4]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.53.54.34:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.53.54.34:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [5]:
strategy = tf.distribute.experimental.TPUStrategy(tpu)
print("Number of accelerators: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Number of accelerators:  8


## Loading the Dataset with the strategy

Here we define a batch size for each replica. We set it to be a multiple of 8 to best leverage the systolic array as defined in the [Google TPU performance guide](https://cloud.google.com/tpu/docs/performance-guide#rule_of_thumb_pick_efficient_values_for_batch_and_feature_dimensions).

In [6]:
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
GLOBAL_BATCH_SIZE

48

### Retrieving the TFRecord dataset

The TFRecord dataset is now entirely processed and ready to be used as input by our training loop. We load it, shuffle it and batch it.

In [10]:
tokenizer = T5Tokenizer.from_pretrained(model_size)
en_de_prefix = tf.reshape(tokenizer("summarize English to German:", return_tensors="tf").input_ids, (-1,))
de_en_prefix = tf.reshape(tokenizer("summarize German to English: ", return_tensors="tf").input_ids, (-1,))
en_en_prefix = tf.reshape(tokenizer("summarize English to English: ", return_tensors="tf").input_ids, (-1,))
de_de_prefix = tf.reshape(tokenizer("summarize German to German: ", return_tensors="tf").input_ids, (-1,))
trans_en_de_prefix = tf.reshape(tokenizer("translate English to German: ", return_tensors="tf").input_ids, (-1,))
trans_de_en_prefix = tf.reshape(tokenizer("translate German to English: ", return_tensors="tf").input_ids, (-1,))

In [11]:
if en_de_prefix[-1] == 1: 
    en_de_prefix = en_de_prefix[:-1]
    de_en_prefix = de_en_prefix[:-1]
    en_en_prefix = en_en_prefix[:-1]
    de_de_prefix = de_de_prefix[:-1]
    trans_en_de_prefix = trans_en_de_prefix[:-1]
    trans_de_en_prefix = trans_de_en_prefix[:-1]


In [12]:
prefix_size = en_de_prefix.shape[0]
assert en_de_prefix.shape[0] == de_en_prefix.shape[0] == en_en_prefix.shape[0] == de_de_prefix.shape[0] == 5
prefix_size

5

In [13]:
en_de_prefix.numpy()

array([21603,  1566,    12,  2968,    10], dtype=int32)

In [15]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'theta-strata-260123'
!gcloud config set project {project_id}

Updated property [core/project].


In [16]:
import numpy as np

bucket = "gs://tpu-bucket-cnn-daily-mail"

def listdir_gcp(directory):
    list_dir = !gsutil ls -r {directory}
    return list_dir

def get_tf_record_files(directory):
    file_list = []
    for item in listdir_gcp(directory):
        if item.split(".")[-1] == "tfrecord":
            file_list.append(str(item))
    return file_list

def get_tfrecord_dataset(directory):
    features = {
        'ger_x': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-prefix_size], tf.int64),
        'ger_x_mask': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-prefix_size], tf.int64),
        'ger_y': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN], tf.int64),
        'ger_y_ids': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN], tf.int64),

        'en_x': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-prefix_size], tf.int64),
        'en_x_mask': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-prefix_size], tf.int64),
        'en_y': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN], tf.int64),
        'en_y_ids': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN], tf.int64),
    }

    tf_records_list = get_tf_record_files(directory)
    print(tf_records_list)
    dataset = tf.data.TFRecordDataset(tf_records_list)

    # Taken from the TensorFlow models repository: https://github.com/tensorflow/models/blob/befbe0f9fe02d6bc1efb1c462689d069dae23af1/official/nlp/bert/input_pipeline.py#L24
    def decode_record(record, features):
        """Decodes a record to a TensorFlow example."""
        example = tf.io.parse_single_example(record, features)

        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.cast(t, tf.int32)
            example[name] = t
        return example


    def select_data_from_record(record):
        i  = tf.random.uniform((1,),0,6,dtype=tf.int32)[0]
        
        if i == 0:
            return tf.concat([de_de_prefix, record['ger_x']], axis=0), tf.concat([tf.ones(prefix_size, dtype=tf.int32), record['ger_x_mask']], axis=0), record['ger_y'], record['ger_y_ids']
        elif i == 1:
            return tf.concat([en_de_prefix, record['en_x']], axis=0), tf.concat([tf.ones(prefix_size, dtype=tf.int32), record['en_x_mask']], axis=0), record['ger_y'], record['ger_y_ids']
        elif i == 2:
            return tf.concat([de_en_prefix, record['ger_x']], axis=0), tf.concat([tf.ones(prefix_size, dtype=tf.int32), record['ger_x_mask']], axis=0), record['en_y'], record['en_y_ids']
        elif i == 3:
            return tf.concat([en_en_prefix, record['en_x']], axis=0), tf.concat([tf.ones(prefix_size, dtype=tf.int32), record['en_x_mask']], axis=0), record['en_y'], record['en_y_ids']
        elif i == 4:
            return tf.concat([trans_de_en_prefix, record['ger_y'], tf.zeros(357, dtype=tf.int32)], axis=0), tf.concat([tf.ones(prefix_size, dtype=tf.int32), record['en_x_mask']], axis=0), record['en_y'], record['en_y_ids']
        else:
            return tf.concat([trans_de_en_prefix, record['en_y'], tf.zeros(357, dtype=tf.int32)], axis=0), tf.concat([tf.ones(prefix_size, dtype=tf.int32), record['en_x_mask']], axis=0), record['ger_y'], record['ger_y_ids']
            
   
    dataset = dataset.map(lambda record: decode_record(record, features))
    dataset = dataset.map(select_data_from_record)
    dataset = dataset.shuffle(500000)
    return dataset.batch(GLOBAL_BATCH_SIZE)



train_dataset = get_tfrecord_dataset(bucket + "/data/cnn_daily_mail_train/")
train_dataset.prefetch(1024)

validation_dataset = get_tfrecord_dataset(bucket + "/data/cnn_daily_mail_val/")
# test_dataset = get_tfrecord_dataset(bucket + "/data/sueddeutsche_val/")


['gs://tpu-bucket-cnn-daily-mail/data/cnn_daily_mail_train/cnn_daily_mail_multilingual-0.tfrecord', 'gs://tpu-bucket-cnn-daily-mail/data/cnn_daily_mail_train/cnn_daily_mail_multilingual-1.tfrecord', 'gs://tpu-bucket-cnn-daily-mail/data/cnn_daily_mail_train/cnn_daily_mail_multilingual-10.tfrecord', 'gs://tpu-bucket-cnn-daily-mail/data/cnn_daily_mail_train/cnn_daily_mail_multilingual-11.tfrecord', 'gs://tpu-bucket-cnn-daily-mail/data/cnn_daily_mail_train/cnn_daily_mail_multilingual-12.tfrecord', 'gs://tpu-bucket-cnn-daily-mail/data/cnn_daily_mail_train/cnn_daily_mail_multilingual-13.tfrecord', 'gs://tpu-bucket-cnn-daily-mail/data/cnn_daily_mail_train/cnn_daily_mail_multilingual-14.tfrecord', 'gs://tpu-bucket-cnn-daily-mail/data/cnn_daily_mail_train/cnn_daily_mail_multilingual-15.tfrecord', 'gs://tpu-bucket-cnn-daily-mail/data/cnn_daily_mail_train/cnn_daily_mail_multilingual-2.tfrecord', 'gs://tpu-bucket-cnn-daily-mail/data/cnn_daily_mail_train/cnn_daily_mail_multilingual-3.tfrecord', 'gs

In [17]:
train_dataset.take(1)

<TakeDataset shapes: ((None, 512), (None, 512), (None, 150), (None, 150)), types: (tf.int32, tf.int32, tf.int32, tf.int32)>

There is an additional step here to distribute the dataset among the different TPU cores. We make use of a strategy method to do so.

Every item held in the dataset (which is a batched dataset) will now be split over the TPU workers. As the TPU we're using has 8 workers and our batch is of size 64, every example will be evenly split in batches of (64 / 8 =) 8 and distributed across workers.

In [18]:
train_dist_dataset = strategy.experimental_distribute_dataset(train_dataset)
validation_dist_dataset = strategy.experimental_distribute_dataset(validation_dataset)

## Model creation

We create a function that will instantiate a new model when called.

In [19]:
def model_fn():
    return TFT5ForConditionalGeneration.from_pretrained(model_size, output_attentions=True)

## Hyperparameters initialization

While in the strategy's scope, we define a sparse categorical crossentropy loss. We define a method `compute_loss` which will be called to compute the loss between the model's prediction and the expected result (or label).

In order to measure the accuracy during training and evaluation, we define two metrics which are both sparse categorical accuracy.

Finally, we initialize a model and create an optimizer object using the Adam optimizer.


In [20]:
with strategy.scope():
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE, from_logits=True)

    def compute_loss(labels, predictions):
        per_example_loss = loss_object(labels, predictions)
        return tf.nn.compute_average_loss(per_example_loss, global_batch_size=GLOBAL_BATCH_SIZE)

    test_loss_metric = tf.keras.metrics.Mean(name='test_loss')
    test_accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

    train_loss_metric = tf.keras.metrics.Mean('training_loss', dtype=tf.float32)
    train_accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy('training_accuracy')
    
    model = model_fn()
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


## Steps

We create two functions that will be called during the training and test steps. 

In [21]:
with strategy.scope():
    def train_step(inputs):
        input_ids, input_mask, y, y_ids = inputs

        with tf.GradientTape() as tape:
            logits = model(input_ids, attention_mask=input_mask, decoder_input_ids=y_ids, training=True)[0]             
            loss = compute_loss(y, logits)

        gradients = tape.gradient(loss, model.trainable_variables)

        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        train_loss_metric.update_state(loss)
        train_accuracy_metric.update_state(y, logits)

    def test_step(inputs):
        input_ids, input_mask, y, y_ids = inputs

        logits = model(input_ids, attention_mask=input_mask, decoder_input_ids=y_ids, training=False)[0]  
        t_loss = compute_loss(y, logits)

        test_loss_metric.update_state(t_loss)
        test_accuracy_metric.update_state(y, logits)

## Training & Evaluation

Finally, using all the previously defined attributes, we create two traced tf.function which will execute the training and test steps in a distributed manner. There is no need for them to return anything as the metrics will directly be updated in the steps described beforehand.

We loop over the number of epochs, training the model and evaluating it at the end of each epoch.

In [22]:
from tqdm.notebook import tqdm


def save_checkpoint(filename, epoch):
    ckpt_file = os.path.join(bucket, "{}/t5_base-{}.ckpt".format(filename, epoch))
    model.save_weights(ckpt_file) 

with strategy.scope():
    @tf.function
    def distributed_train_step(dataset):
        strategy.run(train_step, args=(dataset,))
 

    @tf.function
    def distributed_test_step(dataset):
        strategy.run(test_step, args=(dataset,))


    global_step = 0
    for epoch in range(EPOCHS):
        total_loss = 0.0
        training_steps = 10
        epoch_step = 0
        print_every = 1000

        ### Training loop ###
        for tensor in tqdm(train_dist_dataset, desc="Training"):
            distributed_train_step(tensor)  

            train_loss = train_loss_metric.result().numpy().astype(float)
            train_accuracy = train_accuracy_metric.result().numpy()

            global_step += 1
            epoch_step += 1

            if epoch_step % print_every == 0:
                print(f"Training step {epoch_step} Accuracy: {train_accuracy}, Training loss: {train_loss}")


        ### Test loop ###
        for tensor in tqdm(validation_dist_dataset, desc="Evaluating"):
            distributed_test_step(tensor)
            
        
        ### Output results ###
        test_accuracy = test_accuracy_metric.result().numpy()
        test_loss = test_loss_metric.result().numpy()
        print(f'Epoch: [{epoch}] Validation accuracy: {test_accuracy} Val loss: {test_loss}')

        ### Reset metrics ###
        test_loss_metric.reset_states()
        train_accuracy_metric.reset_states()
        train_loss_metric.reset_states()
        test_accuracy_metric.reset_states()
        epoch_step = 0

        ### savecheckpoint ###
        save_checkpoint("t5_cnn_daily_mail_plus_translation", epoch)

        


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...


Epoch: [5] Validation accuracy: 0.82573401927948 Val loss: 14.963479995727539


Training step 1000 Accuracy: 0.827534556388855, Training loss: 14.901817321777344
Training step 2000 Accuracy: 0.8272059559822083, Training loss: 14.941084861755371
Training step 3000 Accuracy: 0.8274610042572021, Training loss: 14.916778564453125
Training step 4000 Accuracy: 0.8272677659988403, Training loss: 14.931130409240723
Training step 5000 Accuracy: 0.8271967768669128, Training loss: 14.936826705932617




Epoch: [6] Validation accuracy: 0.8273583054542542 Val loss: 14.8106050491333


Training step 1000 Accuracy: 0.828484296798706, Training loss: 14.806118965148926
Training step 2000 Accuracy: 0.828294575214386, Training loss: 14.818527221679688
Training step 3000 Accuracy: 0.8283016681671143, Training loss: 14.817761421203613
Training step 4000 Accuracy: 0.8283092975616455, Training loss: 14.811452865600586
Training step 5000 Accuracy: 0.828520655632019, Training loss: 14.79045295715332




Epoch: [7] Validation accuracy: 0.8282694816589355 Val loss: 14.706399917602539
